# Genie Space Observability E2E Notebook

### Introduction
This notebook covers how to leverage Genie REST APIs and Databricks System Tables to create an e2e Genie Observability workflow, complete with a Genie Observability Dashboard and a Meta-Genie space to ask questions about your Genie spaces!

Running this notebook will create **Two Tables**; _genie_observability_main_table_ and _dbsql_cost_per_query_table_.

- **genie_observability_main_table:** This table consists of ALL the messages sent to your genie spaces in a given workspace along with Genie natural langauge response and the SQL code generated as part of it. There are additional columns that provide the conversation_id, user_feedback, user_email, the statement ID linked to the SQL code and much more.
- **dbsql_cost_per_query_table:** This table consists of the full set of SQL commands that has been executed each linked to a unique statement ID. This table will be useful to filter out those SQL commands executed as part of a Genie space.

### Disclaimer:
- This is a Standalone Notebook, so please ensure the catalog, schema and table names **DO NOT** overwrite any exisitng tables
- The entire solution is vibe coded (with human in the loop!), so please verify and sense check code before running.
- The cost per query attribution code adpated from this materialised view definition - https://github.com/databrickslabs/sandbox/blob/main/dbsql/cost_per_query/PrPr/DBSQL%20Cost%20Per%20Query%20MV%20(PrPr).sql

**Usage Guidance:**
- You can use this notebook as part of the wider Databricks Adoption Dashboard and pipeline that can be deployed as part of Databricks Asset Bundles. Equally, you can download this notebook incorporate it into your own workflow.
- We recommend running this notebook in a test environment first and tailor the code as per your preferance.

**Pre-requisites:**
- This notebook/workflow is targetted towards a Genie space manager
- Entitlements: You must have the Databricks SQL workspace entitlement. See Manage entitlements.
- Compute: CAN USE access on at least one pro or serverless SQL warehouse.
- Data access: SELECT privileges on the data used in the space.
- Genie space ACLs: At least CAN EDIT permissions on the Genie space. Genie space creators automatically have CAN MANAGE permissions on spaces they create. See Genie space ACLs.
- You should have enough permissions to create tables in a schema within a catalog of your choice
- Access to system tables is governed by Unity Catalog. Account admins have access to system tables by default. To allow a user to query system tables, an admin must grant that user USE and SELECT permissions on the system schemas. 




In [0]:
# Upgrade to the latest version of databricks_sdk package
%pip install databricks_sdk --upgrade

In [0]:
# Restart Python to ensure all libraries are reloaded
dbutils.library.restartPython()

In [0]:
# Create a text widget for catalog_name with default value 'users'
dbutils.widgets.text("catalog_name", "users")
# Create a text widget for schema_name with no default value
dbutils.widgets.text("schema_name", "")

# Retrieve the value of catalog_name from the widget
catalog_name = dbutils.widgets.get("catalog_name")
# Retrieve the value of schema_name from the widget
schema_name = dbutils.widgets.get("schema_name")
# Ensure both catalog_name and schema_name are provided
assert catalog_name and schema_name, "catalog_name and schema_name must be provided"

In [0]:
from databricks.sdk import WorkspaceClient
import pandas as pd
import json
from typing import Dict, Any, List
from datetime import datetime

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Get current user information using the workspace client
current_user = w.current_user.me()
user_name = current_user.user_name

# Print the current user's username
print(f"Current user: {user_name}")

# Print the Databricks workspace host URL
print(f"Workspace: {w.config.host}")

Below we list all Genie spaces in the current workspace

In [0]:
# Initialize an empty list to store Genie space metadata
spaces = []
page_token = None

from databricks.sdk import WorkspaceClient
from datetime import datetime
import pandas as pd

# Initialize the Databricks Workspace client
w = WorkspaceClient()

# Paginate through all Genie spaces using the SDK
while True:
    response = w.genie.list_spaces(page_token=page_token)
    for s in response.spaces:
        # Append relevant space details to the list
        spaces.append({
            "space_id": getattr(s, "space_id", None),
            "name": getattr(s, "title", None),
            "description": getattr(s, "description", None),
            "warehouse_id": getattr(s, "warehouse_id", None)
        })
    # Break if there are no more pages
    if not response.next_page_token or response.next_page_token == "":
        break
    page_token = response.next_page_token

# Convert the list of spaces to a Pandas DataFrame
genie_spaces_pdf = pd.DataFrame(spaces)

In [0]:
# Display the DataFrame containing all Genie spaces metadata
if len(genie_spaces_pdf) > 0:
  display(genie_spaces_pdf)
  print("---------------")
  print(f"Total Genie spaces: {len(genie_spaces_pdf)}")
else:
  print("---------------")
  print("No Genie spaces found.")

This code cell below is where we define the single most important function that allows us to generate the genie_observability_main_table table. You can edit this function as per your requirements or even reverse engineer this to generate newer insights!

#### Databricks Python SDK implementation
For implementation with python requests library see appendix

In [0]:
from typing import Dict, Any, List, Optional
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, IntegerType
from pyspark.sql.functions import col, from_unixtime

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.iam import User

# Cache for user email lookups to prevent redundant API calls
USER_CACHE: Dict[str, str] = {}


def get_genie_observability_table(space_id: str) -> DataFrame:
    """
    Fetches and constructs a Spark DataFrame containing observability data 
    for all messages in a Genie space using the Databricks SDK.
    
    Args:
        space_id: The ID of the Genie space to fetch data from.
        
    Returns:
        A Spark DataFrame with message-level observability data.
    """
    w = WorkspaceClient()
    
    # Get space details
    space = w.genie.get_space(space_id=space_id)
    space_name = space.title or f"Space_{space_id}"
    
    records = []
    
    # List all conversations with pagination
    conversations = _list_all_conversations(w, space_id)
    
    for conv in conversations:
        # List all messages in the conversation with pagination
        messages = _list_all_messages(w, space_id, conv.conversation_id)
        
        for msg in messages:
            record = _extract_message_data(msg, space_id, space_name, w)
            records.append(record)
    
    # Create Spark DataFrame
    spark = SparkSession.builder.getOrCreate()
    schema = _get_schema()
    
    if not records:
        return spark.createDataFrame([], schema)
    
    # Convert dicts to tuples in schema field order
    ordered_data = [
        tuple(r.get(field.name) for field in schema.fields)
        for r in records
    ]
        
    df = spark.createDataFrame(ordered_data, schema=schema)
    
    # Add human-readable datetime columns
    df = (df
        .withColumn("created_datetime", from_unixtime(col("created_timestamp") / 1000).cast(TimestampType()))
        .withColumn("last_updated_datetime", from_unixtime(col("last_updated_timestamp") / 1000).cast(TimestampType()))
        .orderBy("created_timestamp")
    )
    
    return df


def _list_all_conversations(w: WorkspaceClient, space_id: str) -> List:
    """Fetches all conversations in a space, handling pagination."""
    conversations = []
    page_token = None
    
    while True:
        response = w.genie.list_conversations(
            space_id=space_id,
            include_all=True,
            page_token=page_token
        )
        conversations.extend(response.conversations or [])
        
        page_token = response.next_page_token
        if not page_token:
            break
    
    return conversations


def _list_all_messages(w: WorkspaceClient, space_id: str, conversation_id: str) -> List:
    """Fetches all messages in a conversation, handling pagination."""
    messages = []
    page_token = None
    
    while True:
        response = w.genie.list_conversation_messages(
            space_id=space_id,
            conversation_id=conversation_id,
            page_token=page_token
        )
        messages.extend(response.messages or [])
        
        page_token = response.next_page_token
        if not page_token:
            break
    
    return messages

def _extract_message_data(message, space_id: str, space_name: str, w: WorkspaceClient) -> Dict[str, Any]:
    """Extracts relevant fields from a GenieMessage into a flat dictionary."""
    msg_dict = message.as_dict()
    
    # Extract IDs (API has both 'id' and 'message_id' for legacy compatibility)
    msg_id = msg_dict.get('message_id') or msg_dict.get('id')
    user_id = msg_dict.get('user_id')
    user_email = _resolve_user_email(str(user_id) if user_id else None, w)

    record = {
        'space_id': space_id,
        'space_name': space_name,
        'message_id': msg_id,
        'conversation_id': msg_dict.get('conversation_id'),
        'user_id': str(user_id) if user_id else None,
        'user_email': user_email,
        'status': str(msg_dict.get('status')) if msg_dict.get('status') else None,
        'created_timestamp': msg_dict.get('created_timestamp'),
        'last_updated_timestamp': msg_dict.get('last_updated_timestamp'),
        'user_question': msg_dict.get('content'),
    }
    
    # Process attachments
    ai_responses, sql_queries, statement_ids, suggested_qs = [], [], [], []
    attachments = msg_dict.get('attachments') or []
    
    for att in attachments:
        # Text attachment
        if text_obj := att.get('text'):
            ai_responses.append(text_obj.get('content', ''))
        
        # Query attachment
        if query_obj := att.get('query'):
            sql_queries.append(query_obj.get('query', ''))
            if stmt_id := query_obj.get('statement_id'):
                statement_ids.append(str(stmt_id))
        
        # Suggested questions attachment
        if sq_obj := att.get('suggested_questions'):
            suggested_qs.extend(sq_obj.get('questions', []))

    def join_non_empty(items: List, sep: str = ' | ') -> Optional[str]:
        filtered = list(filter(None, items))
        return sep.join(filtered) if filtered else None

    record.update({
        'ai_response': join_non_empty(ai_responses),
        'sql_query': join_non_empty(sql_queries),
        'statement_id': join_non_empty(statement_ids),
        'suggested_questions': join_non_empty(suggested_qs, ', '),
        'num_attachments': len(attachments),
    })
    
    # Feedback rating
    feedback = msg_dict.get('feedback') or {}
    record['feedback_rating'] = str(feedback.get('rating')) if feedback.get('rating') else 'NONE'
    
    # Error info
    error = msg_dict.get('error')
    if error and isinstance(error, dict):
        record['error_type'] = error.get('type')
        record['error_message'] = error.get('message') or error.get('error')
    elif error:
        record['error_type'] = 'Unknown'
        record['error_message'] = str(error)
    else:
        record['error_type'] = None
        record['error_message'] = None
    
    return record

def _resolve_user_email(user_id: Optional[str], w: WorkspaceClient) -> Optional[str]:
    """Resolves a user ID to an email address using the SCIM Users API."""
    if not user_id or user_id in ("None", "0"):
        return None
    
    if user_id in USER_CACHE:
        return USER_CACHE[user_id]
    
    try:
        user: User = w.users.get(user_id)
        email = user.user_name
        USER_CACHE[user_id] = email
        return email
    except Exception:
        # Return a placeholder if user lookup fails
        return f"ID_{user_id}"

def _get_schema() -> StructType:
    return StructType([
        StructField("space_id", StringType(), True),
        StructField("space_name", StringType(), True),
        StructField("message_id", StringType(), True),
        StructField("conversation_id", StringType(), True),
        StructField("user_id", StringType(), True),
        StructField("user_email", StringType(), True),
        StructField("status", StringType(), True),
        StructField("created_timestamp", LongType(), True),
        StructField("last_updated_timestamp", LongType(), True),
        StructField("user_question", StringType(), True),
        StructField("ai_response", StringType(), True),
        StructField("sql_query", StringType(), True),
        StructField("statement_id", StringType(), True),
        StructField("suggested_questions", StringType(), True),
        StructField("num_attachments", IntegerType(), True),
        StructField("feedback_rating", StringType(), True),
        StructField("error_type", StringType(), True),
        StructField("error_message", StringType(), True),
    ])


You can test the function by running the function on a Genie space

In [0]:
# --- Usage Example ---

# Define your Space ID
space_id = "XXX" # Replace with your actual Space ID

# Run the function
df = get_genie_observability_table(space_id)

# Display results
display(df)

Below is a simple scripts that runs this function on ALL your genie spaces (currently limited to 10). 

Note: You may not have permssion to view messages of certain Genie spaces as you may not have access to the underlying tables. In this case you will see a lot of NULLs

In [0]:
from databricks.sdk import WorkspaceClient

# Initialize the Databricks workspace client
w = WorkspaceClient()

# Fetch all Genie spaces
print("🔍 Fetching all Genie spaces...")
spaces = []
page_token = None

while True:
    response = w.genie.list_spaces(page_token=page_token)
    for s in response.spaces:
        spaces.append({
            "space_id": getattr(s, "space_id", None),
            "name": getattr(s, "title", None),
            "description": getattr(s, "description", None),
            "warehouse_id": getattr(s, "warehouse_id", None)
        })
    if not response.next_page_token or response.next_page_token == "":
        break
    page_token = response.next_page_token

print(f"✓ Found {len(spaces)} Genie spaces")

# Limit to 10 spaces
MAX_SPACES = 10
if len(spaces) > MAX_SPACES:
    print(f"⚠ Limiting processing to first {MAX_SPACES} spaces\n")
    spaces = spaces[:MAX_SPACES]
else:
    print()

# Collect observability data from all spaces
all_dfs = []

for i, space in enumerate(spaces, 1):
    space_id = space['space_id']
    space_name = space['name']
    
    print(f"\n{'='*80}")
    print(f"[{i}/{len(spaces)}] Processing Space: {space_name}")
    print(f"Space ID: {space_id}")
    print(f"{'='*80}")
    
    try:
        # Get observability data for this space
        df_space = get_genie_observability_table(space_id)
        
        if df_space.count() > 0:
            all_dfs.append(df_space)
            print(f"\n✓ Successfully extracted {df_space.count()} messages from {space_name}")
        else:
            print(f"\n⚠ No messages found in {space_name}")
            
    except Exception as e:
        print(f"\n❌ Error processing space {space_name}: {str(e)}")
        continue

# Combine all DataFrames
if all_dfs:
    print(f"\n\n{'='*80}")
    print("📊 COMBINING ALL RESULTS")
    print(f"{'='*80}")
    
    # Union all DataFrames
    df = all_dfs[0]
    for df_next in all_dfs[1:]:
        df = df.union(df_next)
    
    total_messages = df.count()
    total_spaces = df.select("space_id").distinct().count()
    
    print(f"\n✅ SUCCESS!")
    print(f"   Total Spaces Processed: {total_spaces}")
    print(f"   Total Messages Extracted: {total_messages}")
    print(f"\n{'='*80}\n")
    
    display(df)
else:
    print("\n⚠ No data found across any Genie spaces")
    df = None

Save the table as a delta table and add a nice detailed table description!

In [0]:
# Write the observability data to a Delta table
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(f"{catalog_name}.{schema_name}.genie_observability_main_table")

# Add a detailed table description
spark.sql(f"""
  COMMENT ON TABLE {catalog_name}.{schema_name}.genie_observability_main_table IS 
  'Comprehensive observability table for Genie AI/BI spaces. Contains detailed message-level data including user questions, AI responses, generated SQL queries, execution metadata, user feedback ratings, and error information. Used for monitoring Genie usage, analyzing query patterns, and tracking user engagement across all accessible Genie spaces.'
""")

We successfully created the **genie_observability_main_table** table! Now lets proceed to create the **dbsql_cost_per_query_table** table

**An Aside: How we approached attributing costs from a Genie Space.**

The Strategy: 
- We first run a SQL comman that returns a table containing all SQL queries executed across all workspaces along with the cost attributed to each query. This allows us to understand how much each query actually costed.
- Each query in this table will have a statement ID unique to that query.
- During cost analysis, we can augment the genie_observability_main_table table by performing a left join on statement_id to get a final table ready for cost analysis

The SQL code below is an adaptation of the Materialised View definition. For the MV please see dbsql_cost_per_query_mv.sql file in this folder


In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.{schema_name}.dbsql_cost_per_query_table 
PARTITIONED BY (query_start_hour, workspace_id)
AS
WITH 
-- Must make sure the time window the MV is built on has enough data from ALL 3 tables to generate accurate results (both starts and end time ranges)
table_boundaries AS (
SELECT 
(SELECT MAX(event_time) FROM system.compute.warehouse_events) AS max_events_ts,
(SELECT MAX(end_time) FROM system.query.history) AS max_query_end_ts,
(SELECT MAX(usage_end_time) FROM system.billing.usage) AS max_billing_ts,
(SELECT MIN(event_time) FROM system.compute.warehouse_events) AS min_event_ts,
(SELECT MIN(start_time) FROM system.query.history) AS min_query_start_ts,
(SELECT MIN(usage_end_time) FROM system.billing.usage) AS min_billing_ts,
date_trunc('HOUR', LEAST(max_events_ts, max_query_end_ts, max_billing_ts)) AS selected_end_time,
(date_trunc('HOUR', GREATEST(min_event_ts, min_query_start_ts, min_billing_ts)) + INTERVAL 1 HOUR)::timestamp AS selected_start_time
),

----===== Warehouse Level Calculations =====-----
cpq_warehouse_usage AS (
  SELECT
    usage_metadata.warehouse_id AS warehouse_id,
    *
  FROM
    system.billing.usage AS u
  WHERE
    usage_metadata.warehouse_id IS NOT NULL
    AND usage_start_time >= (SELECT MIN(selected_start_time) FROM table_boundaries)
    AND usage_end_time <= (SELECT MAX(selected_end_time) FROM table_boundaries)
),

prices AS (
  select coalesce(price_end_time, date_add(current_date, 1)) as coalesced_price_end_time, *
  from system.billing.list_prices
  where currency_code = 'USD'
),

filtered_warehouse_usage AS (
    -- Warehouse usage is aggregated hourly, that will be the base assumption and grain of allocation moving forward. 
    -- Assume no duplicate records
    SELECT 
      u.warehouse_id warehouse_id,
      date_trunc('HOUR',u.usage_start_time) AS usage_start_hour,
      date_trunc('HOUR',u.usage_end_time) AS usage_end_hour,
      u.usage_quantity AS dbus,
      (
        CAST(p.pricing.effective_list.default AS FLOAT) * dbus
      ) AS usage_dollars
    FROM
      cpq_warehouse_usage AS u
        left join prices as p
        on u.sku_name=p.sku_name
        and u.usage_unit=p.usage_unit
        and (u.usage_end_time between p.price_start_time and p.coalesced_price_end_time)
),

table_bound_expld AS 
(
select timestampadd(hour, h, selected_start_time) as selected_hours
  from table_boundaries
  join lateral explode(sequence(0, timestampdiff(hour, selected_start_time, selected_end_time), 1)) as t (h)
),

----===== Query Level Calculations =====-----
cpq_warehouse_query_history AS (
  SELECT
    account_id,
    workspace_id,
    statement_id,
    executed_by,
    statement_text,
    compute.warehouse_id AS warehouse_id,
    execution_status,
    COALESCE(client_application, 'Unknown') AS client_application,
    (COALESCE(CAST(total_task_duration_ms AS FLOAT) / 1000, 0) +
      COALESCE(CAST(result_fetch_duration_ms AS FLOAT) / 1000, 0) +
      COALESCE(CAST(compilation_duration_ms AS FLOAT) / 1000, 0)
    )  AS query_work_task_time,
    start_time,
    end_time,
    timestampadd(MILLISECOND , coalesce(waiting_at_capacity_duration_ms, 0) + coalesce(waiting_for_compute_duration_ms, 0) + coalesce(compilation_duration_ms, 0), start_time) AS query_work_start_time,
    timestampadd(MILLISECOND, coalesce(result_fetch_duration_ms, 0), end_time) AS query_work_end_time,
    -- NEW - Query source
    CASE
      WHEN query_source.job_info.job_id IS NOT NULL THEN 'JOB'
      WHEN query_source.legacy_dashboard_id IS NOT NULL THEN 'LEGACY DASHBOARD'
      WHEN query_source.dashboard_id IS NOT NULL THEN 'AI/BI DASHBOARD'
      WHEN query_source.alert_id IS NOT NULL THEN 'ALERT'
      WHEN query_source.notebook_id IS NOT NULL THEN 'NOTEBOOK'
      WHEN query_source.sql_query_id IS NOT NULL THEN 'SQL QUERY'
      WHEN query_source.genie_space_id IS NOT NULL THEN 'GENIE SPACE'
      WHEN client_application IS NOT NULL THEN client_application
      ELSE 'UNKNOWN'
    END AS query_source_type,
    COALESCE(
      query_source.job_info.job_id,
      query_source.legacy_dashboard_id,
      query_source.dashboard_id,
      query_source.alert_id,
      query_source.notebook_id,
      query_source.sql_query_id,
      query_source.genie_space_id,
      'UNKNOWN'
    ) AS query_source_id
  FROM
    system.query.history AS h
  WHERE
    statement_type IS NOT NULL
    -- If query touches the boundaries at all, we will divy it up
    AND start_time < (SELECT selected_end_time FROM table_boundaries)
    AND end_time > (SELECT selected_start_time FROM table_boundaries)
    AND total_task_duration_ms > 0 --exclude metadata operations
     and compute.warehouse_id is not null -- = 'd13162f928a069c7'
)
  ,  cte_warehouse as
(
  select warehouse_id, min(query_work_start_time) as min_start_time
    from cpq_warehouse_query_history
group by warehouse_id
)
,
--- Warehouse + Query Level level allocation
window_events AS (
    SELECT
        warehouse_id,
        event_type,
        event_time,
        cluster_count AS cluster_count,
        CASE
            WHEN cluster_count = 0 THEN 'OFF'
            WHEN cluster_count > 0 THEN 'ON'
        END AS warehouse_state
    FROM system.compute.warehouse_events AS we
    -- Only get window events for when we have query history, otherwise, not usable
    WHERE warehouse_id in (SELECT warehouse_id FROM cte_warehouse)
    AND event_time >= (SELECT timestampadd(day, -1, selected_start_time) FROM table_boundaries)
    AND event_time <= (SELECT selected_end_time FROM table_boundaries)
)
  ,  cte_agg_events_prep as
(
select warehouse_id
      , warehouse_state
      , event_time
      , row_number() over W1
      - row_number() over W2 as grp
  from window_events
window W1 as (partition by warehouse_id                  order by event_time asc)
      , W2 as (partition by warehouse_id, warehouse_state order by event_time asc)
)
  ,  cte_agg_events as
(
  select warehouse_id
       , warehouse_state                                           as window_state
       , min(event_time)                                           as event_window_start
       , lead(min(event_time), 1, selected_end_time) over W as event_window_end
    from cte_agg_events_prep
    join table_boundaries
group by warehouse_id
       , warehouse_state
       , grp
       , selected_end_time
  window W as (partition by warehouse_id order by min(event_time) asc)
)
  ,  cte_all_events as
(
select warehouse_id
     , window_state
     , date_trunc('second', event_window_start) as event_window_start
     , date_trunc('second', event_window_end  ) as event_window_end
  from cte_agg_events
 where date_trunc('second', event_window_start) < date_trunc('second', event_window_end)
 --and date_trunc('second', event_window_start) >= timestamp '2024-11-14 09:00:00'
)
  ,  cte_queries_event_cnt as
(
  select warehouse_id
       , case num
           when 1
           then date_trunc('second', query_work_start_time)
           else timestampadd(second, case when date_trunc('second', query_work_start_time) = date_trunc('second', query_work_end_time) then 1 else 0 end, date_trunc('second', query_work_end_time))
         end as query_event_time
       , sum(num) as num_queries
    from cpq_warehouse_query_history
    join lateral explode(array(1, -1)) as t (num)
group by 1, 2
)
  ,  cte_raw_history as
(
select warehouse_id
     , query_event_time                                           as query_start
     , lead(query_event_time, 1, selected_end_time) over W as query_end
     , sum(num_queries) over W as queries_active
  from cte_queries_event_cnt
  join table_boundaries
window W as (partition by warehouse_id order by query_event_time asc)
)
  ,  cte_raw_history_byday as
(
  select /*+ repartition(64, warehouse_id, query_start_dt) */
         warehouse_id
       , case num
           when 0
           then query_start
           else timestampadd(day, num, query_start::date)
         end::date as query_start_dt
       , case num
           when 0
           then query_start
           else timestampadd(day, num, query_start::date)
         end as query_start
       , case num
           when timestampdiff(day, query_start::date, query_end::date)
           then query_end
           else timestampadd(day, num + 1, query_start::date)
         end as query_end
       , queries_active
    from cte_raw_history
    join lateral explode(sequence(0, timestampdiff(day, query_start::date, query_end::date), 1)) as t (num)
)
  ,  cte_all_time_union as
(
select warehouse_id
     , case num when 1 then event_window_start else event_window_end end ts_start
  from cte_all_events
  join lateral explode(array(1, -1)) as t (num)
 union 
select warehouse_id
     , case num when 1 then query_start else query_end end
  from cte_raw_history_byday
  join lateral explode(array(1, -1)) as t (num)
 union
select warehouse_id, selected_hours
  from cte_warehouse
  join table_bound_expld on true
-- where selected_hours >= timestampadd(day, -1, min_start_time)
)
  ,  cte_periods as
(
select /*+ repartition(64, warehouse_id, dt_start) */
       warehouse_id
     , ts_start::date as dt_start
     , ts_start
     , lead(ts_start, 1, selected_end_time) over W as ts_end
  from cte_all_time_union
  join table_boundaries
window W as (partition by warehouse_id order by ts_start asc)
)
  ,  cte_merge_periods as
(
    select /*+ broadcast(r) */
           p.warehouse_id
         , date_trunc('hour', p.ts_start) as ts_hour
         , sum(timestampdiff(second, p.ts_start, p.ts_end)) as duration
         , case
             when e.window_state = 'OFF'
               or e.window_state is null
             then 'OFF'
             when r.queries_active > 0
             then 'UTILIZED'
             else 'ON_IDLE'
           end as utilization_flag
      from cte_periods           as p
 left join cte_all_events        as e  on e.warehouse_id       = p.warehouse_id
                                      and e.event_window_start < p.ts_end
                                      and e.event_window_end   > p.ts_start
 left join cte_raw_history_byday as r  on r.warehouse_id       = p.warehouse_id
                                      and r.query_start_dt     = p.dt_start
                                      and r.query_start        < p.ts_end
                                      and r.query_end          > p.ts_start
                                      and r.queries_active     > 0
                                      and e.window_state      <> 'OFF'
     where p.ts_start < p.ts_end
  group by all
),

utilization_by_warehouse AS (
  select warehouse_id
       , ts_hour as warehouse_hour
       , coalesce(sum(duration) filter(where utilization_flag = 'UTILIZED'), 0) as utilized_seconds
       , coalesce(sum(duration) filter(where utilization_flag = 'ON_IDLE' ), 0) as idle_seconds
       , coalesce(sum(duration) filter(where utilization_flag = 'OFF'      ), 0) as off_seconds
       , coalesce(sum(duration), 0) as total_seconds
       , try_divide(utilized_seconds, utilized_seconds + idle_seconds)::decimal(3,2) as utilization_proportion
    from cte_merge_periods
group by warehouse_id
       , ts_hour
),

cleaned_warehouse_info AS (
  SELECT
  wu.warehouse_id,
  wu.usage_start_hour AS hour_bucket,
  wu.dbus,
  wu.usage_dollars,
  ut.utilized_seconds,
  ut.idle_seconds,
  ut.total_seconds,
  ut.utilization_proportion
  FROM filtered_warehouse_usage wu
  LEFT JOIN utilization_by_warehouse AS ut ON wu.warehouse_id = ut.warehouse_id -- Join on calculation grain - warehouse/hour
    AND wu.usage_start_hour = ut.warehouse_hour
),

hour_intervals AS (
  -- Generate valid hourly buckets for each query
  SELECT
    statement_id,
    warehouse_id,
    query_work_start_time,
    query_work_end_time,
    query_work_task_time,
    explode(
      sequence(
        0,
        floor((UNIX_TIMESTAMP(query_work_end_time) - UNIX_TIMESTAMP(date_trunc('hour', query_work_start_time))) / 3600)
      )
    ) AS hours_interval,
    timestampadd(hour, hours_interval, date_trunc('hour', query_work_start_time)) AS hour_bucket
  FROM
    cpq_warehouse_query_history
),

statement_proportioned_work AS (
    SELECT * , 
        GREATEST(0,
          UNIX_TIMESTAMP(LEAST(query_work_end_time, timestampadd(hour, 1, hour_bucket))) -
          UNIX_TIMESTAMP(GREATEST(query_work_start_time, hour_bucket))
        ) AS overlap_duration,
        CASE WHEN CAST(query_work_end_time AS DOUBLE) - CAST(query_work_start_time AS DOUBLE) = 0
        THEN 0
        ELSE query_work_task_time * (overlap_duration / (CAST(query_work_end_time AS DOUBLE) - CAST(query_work_start_time AS DOUBLE)))
        END AS proportional_query_work
    FROM hour_intervals
),


attributed_query_work_all AS (
    SELECT
      statement_id,
      hour_bucket,
      warehouse_id,
      SUM(proportional_query_work) AS attributed_query_work
    FROM
      statement_proportioned_work
    GROUP BY
      statement_id,
      warehouse_id,
      hour_bucket
),

--- Cost Attribution
warehouse_time as (
  select
    warehouse_id,
    hour_bucket,
    SUM(attributed_query_work) as total_work_done_on_warehouse
  from
    attributed_query_work_all
  group by
    warehouse_id, hour_bucket
),

-- Create statement_id / hour bucket allocated combinations
history AS (
  SELECT
    a.*,
    b.total_work_done_on_warehouse,
    CASE
      WHEN attributed_query_work = 0 THEN NULL
      ELSE attributed_query_work / total_work_done_on_warehouse
    END AS proportion_of_warehouse_time_used_by_query
  FROM attributed_query_work_all a
    inner join warehouse_time b on a.warehouse_id = b.warehouse_id
              AND a.hour_bucket = b.hour_bucket -- Will only run for completed hours from warehouse usage - nice clean boundary
),

history_with_pricing AS (
  SELECT
    h1.*,
    wh.dbus AS total_warehouse_period_dbus,
    wh.usage_dollars AS total_warehouse_period_dollars,
    wh.utilization_proportion AS warehouse_utilization_proportion,
    wh.hour_bucket AS warehouse_hour_bucket,
    MAX(wh.hour_bucket) OVER() AS warehouse_max_hour_bucket
  FROM
    history AS h1
    LEFT JOIN cleaned_warehouse_info AS wh ON h1.warehouse_id = wh.warehouse_id AND h1.hour_bucket = wh.hour_bucket
),

-- This is at the statement_id / hour grain (there will be duplicates for each statement for each hour bucket the query spans)

query_attribution AS (
  SELECT
    a.*,
    warehouse_max_hour_bucket AS most_recent_billing_hour,
    CASE WHEN warehouse_hour_bucket IS NOT NULL THEN 'Has Billing Record' ELSE 'No Billing Record for this hour and warehouse yet available' END AS billing_record_check,
    CASE
      WHEN total_work_done_on_warehouse = 0 THEN NULL
      ELSE attributed_query_work / total_work_done_on_warehouse
    END AS query_task_time_proportion,

    (warehouse_utilization_proportion * total_warehouse_period_dollars) * query_task_time_proportion  AS query_attributed_dollars_estimation,
    (warehouse_utilization_proportion * total_warehouse_period_dbus) * query_task_time_proportion  AS query_attributed_dbus_estimation
  FROM
    history_with_pricing a
)

-- Final Output
select
      qq.statement_id,
      FIRST(qq.query_source_id) AS query_source_id,
      FIRST(qq.query_source_type) AS query_source_type,
      FIRST(qq.client_application) AS client_application,
      FIRST(qq.executed_by) AS executed_by,
      FIRST(qq.warehouse_id) AS warehouse_id,
      FIRST(qq.statement_text) AS statement_text,
      FIRST(qq.workspace_id) AS workspace_id,
      COLLECT_LIST(NAMED_STRUCT('hour_bucket', qa.hour_bucket, 'hour_attributed_cost', query_attributed_dollars_estimation, 'hour_attributed_dbus', query_attributed_dbus_estimation)) AS statement_hour_bucket_costs,
      FIRST(qq.start_time) AS start_time,
      FIRST(qq.end_time) AS end_time,
      FIRST(qq.query_work_start_time) AS query_work_start_time,
      FIRST(qq.query_work_end_time) AS query_work_end_time,
      COALESCE(timestampdiff(MILLISECOND, FIRST(qq.start_time), FIRST(qq.end_time))/1000, 0) AS duration_seconds,
      COALESCE(timestampdiff(MILLISECOND, FIRST(qq.query_work_start_time), FIRST(qq.query_work_end_time))/1000, 0) AS query_work_duration_seconds,
      FIRST(query_work_task_time) AS query_work_task_time_seconds,
      SUM(query_attributed_dollars_estimation) AS query_attributed_dollars_estimation,
      SUM(query_attributed_dbus_estimation) AS query_attributed_dbus_estimation,
      FIRST(CASE
        WHEN query_source_type = 'JOB' THEN CONCAT('/jobs/', query_source_id)
        WHEN query_source_type = 'SQL QUERY' THEN CONCAT('/sql/queries/', query_source_id)
        WHEN query_source_type = 'AI/BI DASHBOARD' THEN CONCAT('/sql/dashboardsv3/', query_source_id)
        WHEN query_source_type = 'LEGACY DASHBOARD' THEN CONCAT('/sql/dashboards/', query_source_id)
        WHEN query_source_type = 'ALERTS' THEN CONCAT('/sql/alerts/', query_source_id)
        WHEN query_source_type = 'GENIE SPACE' THEN CONCAT('/genie/rooms/', query_source_id)
        WHEN query_source_type = 'NOTEBOOK' THEN CONCAT('/editor/notebooks/', query_source_id)
        ELSE ''
      END) as url_helper,
      FIRST(CONCAT('/sql/history?uiQueryProfileVisible=true&queryId=', qq.statement_id)) AS query_profile_url,
       FIRST(most_recent_billing_hour) AS most_recent_billing_hour,
       FIRST(billing_record_check) AS billing_record_check,
       date_trunc('HOUR', FIRST(qq.start_time)) AS query_start_hour
      from query_attribution qa
      LEFT JOIN cpq_warehouse_query_history AS qq ON qa.statement_id = qq.statement_id -- creating dups of the objects but just re-aggregating
            AND qa.warehouse_id = qq.warehouse_id
      GROUP BY qq.statement_id;""")

In [0]:
display(spark.table(f"{catalog_name}.{schema_name}.dbsql_cost_per_query_table"))

In [0]:
# Add a detailed table description
spark.sql(f"""
  COMMENT ON TABLE {catalog_name}.{schema_name}.dbsql_cost_per_query_table IS 
  'This table contains information about the cost of each query executed in Databricks SQL.'
""")

## Analytics!

With both critical tables created, you can now join them to generate actionable insights! Use these SQL queries as a foundation for custom dashboards and Meta-Genie spaces. For inspiration, explore the example datasets and dashboards available in this folder and navigate to the Genie Drill Down tab - src/dashboards/lh_adoption_dashboard.lvdash.json

Some example SQL queries (recommended by Databricks Assistant!) are below

### Cost Analysis of Genie-Generated Queries: Join both tables to see how much Genie AI queries cost compared to other query sources, and identify the most expensive AI-generated queries.

In [0]:
%sql
SELECT 
    g.space_name,
    g.user_email,
    g.user_question,
    c.query_attributed_dollars_estimation,
    c.query_attributed_dbus_estimation,
    c.start_time,
    c.end_time
FROM kg_test_workspace.default.genie_observability_main_table g
LEFT JOIN kg_test_workspace.default.dbsql_cost_per_query_table c
    ON g.statement_id = c.statement_id
WHERE c.query_attributed_dollars_estimation IS NOT NULL
ORDER BY c.query_attributed_dollars_estimation DESC
LIMIT 20

### Genie Success Rate & Error Analysis: Analyze which Genie spaces have the highest error rates and what types of errors occur most frequently.

In [0]:
%sql
SELECT 
    space_name,
    status,
    error_type,
    COUNT(*) as message_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY space_name), 2) as pct_of_space
FROM kg_test_workspace.default.genie_observability_main_table
GROUP BY space_name, status, error_type
ORDER BY space_name, message_count DESC

### User Engagement & Feedback Patterns: Identify which users are most active with Genie and how they rate the responses.

In [0]:
%sql
SELECT 
    user_email,
    COUNT(DISTINCT conversation_id) as total_conversations,
    COUNT(message_id) as total_messages,
    COUNT(CASE WHEN feedback_rating = 'positive' THEN 1 END) as positive_feedback,
    COUNT(CASE WHEN feedback_rating = 'negative' THEN 1 END) as negative_feedback,
    ROUND(AVG(duration_seconds), 2) as avg_response_time_sec
FROM kg_test_workspace.default.genie_observability_main_table
WHERE user_email IS NOT NULL
GROUP BY user_email
ORDER BY total_messages DESC
LIMIT 20

### Query Performance by Source Type: Compare query performance and costs across different query sources (Genie vs dashboards vs notebooks vs direct queries).

In [0]:
%sql
SELECT 
    query_source_type,
    client_application,
    COUNT(*) as query_count,
    ROUND(AVG(query_attributed_dollars_estimation), 4) as avg_cost_dollars,
    ROUND(SUM(query_attributed_dollars_estimation), 2) as total_cost_dollars,
    ROUND(AVG(TIMESTAMPDIFF(SECOND, start_time, end_time)), 2) as avg_duration_sec
FROM kg_test_workspace.default.dbsql_cost_per_query_table
WHERE query_attributed_dollars_estimation IS NOT NULL
GROUP BY query_source_type, client_application
ORDER BY total_cost_dollars DESC

### Time-Based Usage & Cost Trends: Analyze hourly patterns to identify peak usage times and associated costs for both Genie and overall SQL queries.

In [0]:
%sql
SELECT 
    DATE_TRUNC('hour', c.start_time) as hour_bucket,
    COUNT(DISTINCT c.statement_id) as total_queries,
    COUNT(DISTINCT g.message_id) as genie_queries,
    ROUND(SUM(c.query_attributed_dollars_estimation), 2) as total_cost,
    ROUND(AVG(c.query_attributed_dollars_estimation), 4) as avg_cost_per_query
FROM kg_test_workspace.default.dbsql_cost_per_query_table c
LEFT JOIN kg_test_workspace.default.genie_observability_main_table g
    ON c.statement_id = g.statement_id
WHERE c.start_time >= CURRENT_DATE - INTERVAL 30 DAYS
GROUP BY DATE_TRUNC('hour', c.start_time)
ORDER BY hour_bucket DESC
LIMIT 100

This concludes this notebook. We now have created two critical tables that will help us monitor and analyse our genie spaces across multiple dimensions:
- Cost attribution and chargeback
- Usability and accuracy
- Insights to further improve the space by adding more tables, adding SQL examples, trusted assets etc



### Take this to the next level
You can enhance this notebook by incorporating more advanced techniques such as:
- Run topic modelling on user user questions to identify common patterns that can help you improve youe Genie space
- Incorporate more advanced cost attribution strategies for charge backs.
- Incorporate this notebook as part of your custome Databricks Jobs process or the full observability stack by deploying this repo through Databricks Asset Bundles

### Appendix

REST API Implementation for get_genie_observability_table function using Python Requests library

In [0]:
# import requests
# import json
# from typing import Dict, Any, List
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType, IntegerType
# from pyspark.sql.functions import col, from_unixtime

# # Cache for User Email lookups to prevent redundant API calls
# USER_CACHE = {}

# def get_genie_observability_table(
#     space_id: str, 
#     databricks_token: str, 
#     host_url: str,
#     include_all_users: bool = True
# ) -> 'pyspark.sql.DataFrame':
#     """
#     Fetches and constructs a Spark DataFrame containing observability data for all messages in a Genie space.

#     Args:
#         space_id (str): The Genie space ID.
#         databricks_token (str): Databricks personal access token for authentication.
#         host_url (str): Databricks workspace host URL.
#         include_all_users (bool, optional): Whether to include all users' conversations. Defaults to True.

#     Returns:
#         pyspark.sql.DataFrame: DataFrame with observability records for the specified Genie space.
#     """
#     host_url = host_url.rstrip('/')
#     headers = {'Authorization': f'Bearer {databricks_token}', 'Content-Type': 'application/json'}
    
#     # Step 1: Get space name
#     space_url = f"{host_url}/api/2.0/genie/spaces/{space_id}"
#     space_resp = requests.get(space_url, headers=headers)
#     space_resp.raise_for_status()
#     space_name = space_resp.json().get('title', f"Space_{space_id}")
    
#     # Step 2: Get conversations
#     conversations = _get_all_conversations(space_id, host_url, headers, include_all_users)
    
#     # Step 3: Extract and Flatten
#     records = []
#     for conv in conversations:
#         messages = _get_all_conversation_messages(space_id, conv['conversation_id'], host_url, headers)
#         for msg in messages:
#             record = _extract_message_data(msg, space_id, space_name, host_url, headers)
#             records.append(record)
    
#     # Step 4: Create Spark DataFrame
#     spark = SparkSession.builder.getOrCreate()
#     schema = _get_schema()
    
#     if not records:
#         return spark.createDataFrame([], schema)
        
#     df = spark.createDataFrame(records, schema=schema)
    
#     # Convert timestamps to Datetime
#     df = df.withColumn("created_datetime", from_unixtime(col("created_timestamp") / 1000).cast(TimestampType())) \
#            .withColumn("last_updated_datetime", from_unixtime(col("last_updated_timestamp") / 1000).cast(TimestampType())) \
#            .orderBy("created_timestamp")
    
#     return df

# def _get_all_conversations(space_id: str, host_url: str, headers: Dict[str, str], 
#                            include_all: bool) -> List[Dict[str, Any]]:
#     """
#     Retrieves all conversations for a given Genie space, handling pagination.

#     Args:
#         space_id (str): Genie space ID.
#         host_url (str): Databricks workspace host URL.
#         headers (Dict[str, str]): HTTP headers for authentication.
#         include_all (bool): Whether to include all users' conversations.

#     Returns:
#         List[Dict[str, Any]]: List of conversation metadata dictionaries.
#     """
#     all_conversations = []
#     page_token = None
    
#     while True:
#         url = f"{host_url}/api/2.0/genie/spaces/{space_id}/conversations"
#         params = {'page_size': 100}
        
#         if page_token:
#             params['page_token'] = page_token
#         if include_all:
#             params['include_all'] = 'true'
        
#         response = requests.get(url, headers=headers, params=params)
#         response.raise_for_status()
#         result = response.json()
        
#         conversations = result.get('conversations', [])
#         all_conversations.extend(conversations)
        
#         page_token = result.get('next_page_token')
#         if not page_token:
#             break
    
#     return all_conversations


# def _get_all_conversation_messages(space_id: str, conversation_id: str, 
#                                    host_url: str, headers: Dict[str, str]) -> List[Dict[str, Any]]:
#     """
#     Retrieves all messages from a specific conversation, handling pagination.

#     Args:
#         space_id (str): Genie space ID.
#         conversation_id (str): Conversation ID.
#         host_url (str): Databricks workspace host URL.
#         headers (Dict[str, str]): HTTP headers for authentication.

#     Returns:
#         List[Dict[str, Any]]: List of message dictionaries.
#     """
#     all_messages = []
#     page_token = None
    
#     while True:
#         url = f"{host_url}/api/2.0/genie/spaces/{space_id}/conversations/{conversation_id}/messages"
#         params = {'page_size': 100}
        
#         if page_token:
#             params['page_token'] = page_token
        
#         response = requests.get(url, headers=headers, params=params)
#         response.raise_for_status()
#         result = response.json()
        
#         messages = result.get('messages', [])
#         all_messages.extend(messages)
        
#         page_token = result.get('next_page_token')
#         if not page_token:
#             break
    
#     return all_messages
    
# def _extract_message_data(message: Dict[str, Any], space_id: str, space_name: str, host_url: str, headers: dict) -> Dict[str, Any]:
#     """
#     Extracts and flattens relevant fields from a Genie message object for observability.

#     Args:
#         message (Dict[str, Any]): Message dictionary from Genie API.
#         space_id (str): Genie space ID.
#         space_name (str): Genie space name.
#         host_url (str): Databricks workspace host URL.
#         headers (dict): HTTP headers for authentication.

#     Returns:
#         Dict[str, Any]: Flattened record with observability fields.
#     """
#     # Resolve User Email
#     user_id = str(message.get('user_id')) if message.get('user_id') else None
#     user_email = _resolve_email(user_id, host_url, headers)

#     record = {
#         'space_id': space_id,
#         'space_name': space_name,
#         'message_id': message.get('message_id'),
#         'conversation_id': message.get('conversation_id'),
#         'user_id': user_id,
#         'user_email': user_email,
#         'status': message.get('status'),
#         'created_timestamp': message.get('created_timestamp'),
#         'last_updated_timestamp': message.get('last_updated_timestamp'),
#         'user_question': message.get('content'),
#     }
    
#     ai_responses, sql_queries, statement_ids, suggested_qs = [], [], [], []
    
#     for att in message.get('attachments', []):
#         # Text Attachment
#         if att.get('text'):
#             ai_responses.append(att['text'].get('content', ''))
        
#         # Query Attachment (Corrected sibling path)
#         query_obj = att.get('query')
#         if query_obj:
#             sql_queries.append(query_obj.get('query', ''))
#             s_id = query_obj.get('statement_id')
#             if s_id: statement_ids.append(str(s_id))
            
#         # Suggested Questions
#         if att.get('suggested_questions'):
#             suggested_qs.extend(att['suggested_questions'].get('questions', []))

#     # Helper to return None instead of empty string
#     def join_clean(lst, sep=' | '): return sep.join(filter(None, lst)) if lst else None

#     record.update({
#         'ai_response': join_clean(ai_responses),
#         'sql_query': join_clean(sql_queries),
#         'statement_id': join_clean(statement_ids),
#         'suggested_questions': join_clean(suggested_qs, sep=', '),
#         'num_attachments': len(message.get('attachments', []))
#     })
    
#     # Feedback - Where the "Review Comments" live
#     feedback = message.get('feedback', {})
#     record['feedback_rating'] = feedback.get('rating', 'NONE')
        
#     # Errors
#     error = message.get('error', {})
#     record['error_type'] = error.get('type')
#     record['error_message'] = error.get('error')
    
#     return record

# def _resolve_email(user_id: str, host_url: str, headers: dict) -> str:
#     """
#     Resolves a Databricks user ID to an email address using the SCIM API, with caching.

#     Args:
#         user_id (str): Databricks user ID.
#         host_url (str): Databricks workspace host URL.
#         headers (dict): HTTP headers for authentication.

#     Returns:
#         str: User email if found, or a fallback string.
#     """
#     if not user_id or user_id in ["None", "0"]: return None
#     if user_id in USER_CACHE: return USER_CACHE[user_id]
    
#     try:
#         url = f"{host_url}/api/2.0/preview/scim/v2/Users/{user_id}"
#         resp = requests.get(url, headers=headers)
#         if resp.status_code == 200:
#             email = resp.json().get('userName')
#             USER_CACHE[user_id] = email
#             return email
#     except: pass
#     return f"ID_{user_id}"

# def _get_schema() -> StructType:
#     """
#     Returns the schema for the Genie observability Spark DataFrame.

#     Returns:
#         StructType: Spark schema for observability records.
#     """
#     return StructType([
#         StructField("space_id", StringType(), True),
#         StructField("space_name", StringType(), True),
#         StructField("message_id", StringType(), True),
#         StructField("conversation_id", StringType(), True),
#         StructField("user_id", StringType(), True),
#         StructField("user_email", StringType(), True),
#         StructField("status", StringType(), True),
#         StructField("created_timestamp", LongType(), True),
#         StructField("last_updated_timestamp", LongType(), True),
#         StructField("user_question", StringType(), True),
#         StructField("ai_response", StringType(), True),
#         StructField("sql_query", StringType(), True),
#         StructField("statement_id", StringType(), True),
#         StructField("suggested_questions", StringType(), True),
#         StructField("num_attachments", IntegerType(), True),
#         StructField("feedback_rating", StringType(), True),
#         StructField("error_type", StringType(), True),
#         StructField("error_message", StringType(), True),
#     ])


In [0]:
# # Set your Genie space ID
# space_id = "XXX"  # Replace with your space ID

# # Use WorkspaceClient to get host and token
# from databricks.sdk import WorkspaceClient

# # Initialize the Databricks workspace client
# w = WorkspaceClient()

# # Retrieve the workspace host URL
# host = w.config.host

# # Retrieve the API token from the workspace client config
# token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# # Fetch Genie observability data for the specified space
# df = get_genie_observability_table(space_id, token, host)

# # Display the resulting DataFrame
# display(df)

In [0]:
# from databricks.sdk import WorkspaceClient
# from datetime import datetime
# import pandas as pd

# # Initialize the Databricks workspace client
# w = WorkspaceClient()

# # Get authentication parameters
# token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
# host = w.config.host

# # Fetch all Genie spaces
# print("🔍 Fetching all Genie spaces...")
# spaces = []
# page_token = None

# while True:
#     response = w.genie.list_spaces(page_token=page_token)
#     for s in response.spaces:
#         spaces.append({
#             "space_id": getattr(s, "space_id", None),
#             "name": getattr(s, "title", None),
#             "description": getattr(s, "description", None),
#             "warehouse_id": getattr(s, "warehouse_id", None)
#         })
#     if not response.next_page_token or response.next_page_token == "":
#         break
#     page_token = response.next_page_token

# print(f"✓ Found {len(spaces)} Genie spaces")

# # Limit to 10 spaces
# MAX_SPACES = 10
# if len(spaces) > MAX_SPACES:
#     print(f"⚠ Limiting processing to first {MAX_SPACES} spaces\n")
#     spaces = spaces[:MAX_SPACES]
# else:
#     print()

# # Collect observability data from all spaces
# all_dfs = []

# for i, space in enumerate(spaces, 1):
#     space_id = space['space_id']
#     space_name = space['name']
    
#     print(f"\n{'='*80}")
#     print(f"[{i}/{len(spaces)}] Processing Space: {space_name}")
#     print(f"Space ID: {space_id}")
#     print(f"{'='*80}")
    
#     try:
#         # Get observability data for this space
#         df_space = get_genie_observability_table(space_id, token, host)
        
#         if df_space.count() > 0:
#             all_dfs.append(df_space)
#             print(f"\n✓ Successfully extracted {df_space.count()} messages from {space_name}")
#         else:
#             print(f"\n⚠ No messages found in {space_name}")
            
#     except Exception as e:
#         print(f"\n❌ Error processing space {space_name}: {str(e)}")
#         continue

# # Combine all DataFrames
# if all_dfs:
#     print(f"\n\n{'='*80}")
#     print("📊 COMBINING ALL RESULTS")
#     print(f"{'='*80}")
    
#     # Union all DataFrames
#     df = all_dfs[0]
#     for df_next in all_dfs[1:]:
#         df = df.union(df_next)
    
#     total_messages = df.count()
#     total_spaces = df.select("space_id").distinct().count()
    
#     print(f"\n✅ SUCCESS!")
#     print(f"   Total Spaces Processed: {total_spaces}")
#     print(f"   Total Messages Extracted: {total_messages}")
#     print(f"\n{'='*80}\n")
    
#     display(df)
# else:
#     print("\n⚠ No data found across any Genie spaces")
#     df = None